In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time
from functools import reduce

In [2]:
products=2
coef_p1 = [[-6,0.01,1,-0.003,23],
           [-6,0.01,1,-0.003,23],
           [-6,0.01,1,-0.003,23]]

coef_p2 = [[1.2,-0.004,-5,0.02,20],
           [1.2,-0.004,-5,0.02,20],
           [1.2,-0.004,-5,0.02,20]]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]
base_price = [base_price_1,base_price_2]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
flag_p1=[1,0,0]
flag_p2=[0,0,1]




target_trade_spend = [276860,223573]
total_target_trade_spend_final = sum(target_trade_spend)

target_edlp_spend = [63366,35525]
target_tpr_spend = [213494,188048]

idx = range(6)
products = 2
weeks = 3
tot = products*weeks

In [3]:
target_trade_spend_lower_bd = [x * 0.9 for x in target_trade_spend]
target_trade_spend_upper_bd = [x * 1.1 for x in target_trade_spend]

target_edlp_spend_lower_bd = [x * 0.8 for x in target_edlp_spend]
target_edlp_spend_upper_bd = [x * 1.2 for x in target_edlp_spend]

target_tpr_spend_lower_bd = [x * 0.8 for x in target_tpr_spend]
target_tpr_spend_upper_bd = [x * 1.2 for x in target_tpr_spend]

In [4]:
import re
def get_info_from_results(results, info_string):
    i = str(results).lower().find(info_string.lower()) + len(info_string)
    value = ''
    while str(results)[i] != '\n':
        value = value + str(results)[i]
        i += 1
    return float(value)
def get_obj_frm_res(results, info_string):
#     print(results)
    results = results.lower()
    info_string = info_string.lower()
    z = re.search(info_string, results).start()
    
    return(float(results[z+108:z+125]))

In [5]:
def cal_sales_prior(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (log( edlp[p,i])*coef[i][cc])* z
                tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*z)*(1- z)
            else:
                edlp_sum+= (log( edlp[p,i])*coef[i][cc])* flag[p,i]
                tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*coef[i][cc+1])*(1- flag[p,i])
                
            cc+=2
            
        unit_sales.append(exp(edlp_sum+ tpr_sum +coef[i][-1]))                 
    return unit_sales

def cal_sales(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
            edlp_sum+= (log( edlp[p,i])*coef[i][cc])* flag[p,i]
            tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*coef[i][cc+1])*(1- flag[p,i])
            cc+=2
        unit_sales.append(exp(edlp_sum+ tpr_sum +coef[i][-1]))                 
   
    return unit_sales

def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j,i]/100)*(1-flag[j,i]))  +  (edlp[j,i]*flag[j,i]))
                   
    return price
                   
def calc_dollar_sales(edlp,tpr,flag,coef, prior=False):
    dollar_sales=[]
    for j in range(products):
        if not prior:
            unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
            price = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales = cal_sales_prior(edlp,tpr,flag,coef[j],j)
            price = base_price[j]

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales


def calc_total_trade_spent(edlp,tpr,flag,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp,tpr,flag,coef):
    edlp_trade_spent = []
#     flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j,i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent

def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

In [6]:
def create_model(init):
    print("Init value= ",init)
    def initial(model,i,j):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    model.weeks = Param(initialize=3, domain=PositiveIntegers)
    model.ppgs = Param(initialize=2, domain=PositiveIntegers)
    model.wk_index = RangeSet(0, model.weeks - 1)
    model.ppg_index = RangeSet(0, model.ppgs - 1)

    model.edlp = Var(model.ppg_index, model.wk_index, initialize=9.5, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr = Var(model.ppg_index, model.wk_index, initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag = Var(model.ppg_index, model.wk_index, initialize=initial, domain=Binary)
    
    prior_prom_sales = calc_dollar_sales(model.edlp,model.tpr,model.flag,coef, True)
    dollar_sales = calc_dollar_sales(model.edlp,model.tpr,model.flag,coef)
    total_trade_spent = calc_total_trade_spent(model.edlp,model.tpr,model.flag,coef)
    edlp_trade_spent = calc_edlp_trade_spent(model.edlp,model.tpr,model.flag,coef)
    
    lift = calculate_lift(dollar_sales, prior_prom_sales)
    
    model.obj = Objective(expr=sum(lift) ,sense= maximize)
    model.c1 = Constraint(expr = sum(total_trade_spent[0]) + sum(total_trade_spent[1]) == total_target_trade_spend_final)
    
    for i in range(products):
        name_1 = 'constraint_lb_total' + str(i)
        name_2 = 'constraint_ub_total' + str(i)
        name_3 = 'constraint_lb_edlp' + str(i)
        name_4 = 'constraint_ub_edlp' + str(i)
        
        model.add_component(name_1, Constraint(expr = sum(total_trade_spent[i]) >= target_trade_spend_lower_bd[i]))
        model.add_component(name_2, Constraint(expr = sum(total_trade_spent[i]) <= target_trade_spend_upper_bd[i]))
        
        model.add_component(name_3, Constraint(expr = sum(edlp_trade_spent[i]) >= target_edlp_spend_lower_bd[i]))
        model.add_component(name_4, Constraint(expr = sum(edlp_trade_spent[i]) <= target_edlp_spend_upper_bd[i]))
    
#     model.c2 = Constraint(expr = sum(total_trade_spent[0]) >= target_trade_spend_lower_bd[0])
#     model.c3 = Constraint(expr = sum(total_trade_spent[0]) <= target_trade_spend_upper_bd[0])
    
#     model.c4 = Constraint(expr = sum(total_trade_spent[1]) >= target_trade_spend_lower_bd[1])
#     model.c5 = Constraint(expr = sum(total_trade_spent[1]) <= target_trade_spend_upper_bd[1])
    
    
#     model.c6 = Constraint(expr = sum(edlp_trade_spent[0]) >= target_edlp_spend_lower_bd[0])
#     model.c7 = Constraint(expr = sum(edlp_trade_spent[0]) <= target_edlp_spend_upper_bd[0])
    
#     model.c8 = Constraint(expr = sum(edlp_trade_spent[1]) >= target_edlp_spend_lower_bd[1])
#     model.c9 = Constraint(expr = sum(edlp_trade_spent[1]) <= target_edlp_spend_upper_bd[1])
    
#     model.c1 = Constraint(expr = sum(total_trade_spent[0]) == target_trade_spend[0])
#     model.c2 = Constraint(expr = sum(total_trade_spent[1]) == target_trade_spend[1])
#     model.c3 = Constraint(expr = sum(edlp_trade_spent[0]) == target_edlp_spend[0] )
#     model.c4 = Constraint(expr = sum(edlp_trade_spent[1]) == target_edlp_spend[1] )
#     model.c5 = Constraint(expr = sum(calc_flag_util(model.flag))==0 )
    return model

In [7]:
def call_solver(init, name='bonmin'):
    assert type(init) == list
    model = create_model(init)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    t = get_info_from_results(results_1, 'Time: ')
    print('Time taken is: ', t)
    model.display('hola.txt')
    file =  open('hola.txt', 'r')
    data = file.read()
    o = get_obj_frm_res(data, 'Objectives')
    file.close()
    print('Objective is: ', o)
    print('-------------------------------------------------')
    return o,t,model

combinations=[[0,0,0,0,0,0],[0,0,0,0,0,1],[0,0,0,0,1,1],[0,0,0,1,1,1],
             [1,0,0,0,0,0],[1,0,0,0,0,1],[1,0,0,0,1,1],[1,0,0,1,1,1],[1,1,0,0,0,0],[1,1,0,0,0,1],
             [1,1,0,0,1,1],[1,1,0,1,1,1],[1,1,1,0,0,0],[1,1,1,0,0,1],[1,1,1,0,1,1],[1,1,1,1,1,1]]    

time_lst = []
obj_lst = []
time_lst_1 = []
for c in combinations:
    start_time = time.time()
    t_lst, o_lst,model = call_solver(c, 'bonmin')
    end_time = time.time()
    time_lst.append(t_lst)
    obj_lst.append(o_lst)
    time_lst_1.append(end_time - start_time)
    model.display()
    break
# time_lst, obj_lst
# 5566396.813079901

Init value=  [0, 0, 0, 0, 0, 0]
Time taken is:  0.9887030124664307
Objective is:  643501.7147461211
-------------------------------------------------
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 :      9.7500000025 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.670215209060805 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.670215209060805 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.533860949684271 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.877996723916484 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.877996723916484 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=tpr_index
        Key    : Lower : Value              : Upper : Fixed : Stale : Domain
        (0, 0) :     5 :  16.31710842740815 :    50 : False : False : NonNegati